In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools


In [2]:
from google.colab import drive


In [3]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:

CUDA = torch.cuda.is_available()

In [5]:
CUDA

True

In [6]:
device = torch.device('cuda' if CUDA else 'cpu')
device

device(type='cuda')

In [7]:
os.chdir('/content/drive/My Drive/cornell_movie_dialogs_corpus')

In [8]:
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

In [9]:
with open(lines_filepath, encoding="utf8", errors='ignore') as file:
  lines = file.readlines()
for line in lines[:8]:
  print(line.strip())    #u0 is character 0 in movie m 

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [10]:
with open(conv_filepath, encoding="utf8", errors='ignore') as cfile:
  linesNew = cfile.readlines()
for line1 in linesNew[:8]:
  print(line1.strip())    #u0 is character 0 in movie m 

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']


In [11]:
line_fields = ["lineID", "characterID", "movieID", "character", "text"]
lines = {}
with open(lines_filepath, mode = 'r', encoding='iso-8859-1') as f:
  for line in f:
    values = line.split(" +++$+++ ")
    lineObj = {}
    for i, field in enumerate(line_fields):
      lineObj[field] = values[i]
    lines[lineObj['lineID']] = lineObj  

In [12]:
lines

{'L1045': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L1045',
  'movieID': 'm0',
  'text': 'They do not!\n'},
 'L1044': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L1044',
  'movieID': 'm0',
  'text': 'They do to!\n'},
 'L985': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L985',
  'movieID': 'm0',
  'text': 'I hope so.\n'},
 'L984': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L984',
  'movieID': 'm0',
  'text': 'She okay?\n'},
 'L925': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L925',
  'movieID': 'm0',
  'text': "Let's go.\n"},
 'L924': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L924',
  'movieID': 'm0',
  'text': 'Wow\n'},
 'L872': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L872',
  'movieID': 'm0',
  'text': "Okay -- you're gonna need to learn how to lie.\n"},
 'L871': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L871',
  'movieID': 'm0',
  'text': 'No

In [13]:
conv_fields = ["character1ID", "character2ID", "movieID", "utteranceIDs"]
conversations = []
with open(conv_filepath, mode = 'r', encoding = 'iso-8859-1') as f:
  for line in f:
    values = line.split(" +++$+++ ")
    convObj = {}
    for i, field in enumerate(conv_fields):
      convObj[field] = values[i]
    #convObj["utteranceIDs"] are "['L213', 'L2312']" etc
    lineIds = eval(convObj["utteranceIDs"])  #string op to list
    convObj['lines'] = []
    for lineId in lineIds:
      convObj['lines'].append(lines[lineId])
    conversations.append(convObj)  


In [14]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'lines': [{'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L194',
   'movieID': 'm0',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L195',
   'movieID': 'm0',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L196',
   'movieID': 'm0',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L197',
   'movieID': 'm0',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}],
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n"}

In [15]:
qa_pairs = []
for conversation in conversations:
  for i in range(len(conversation["lines"]) - 1):
    question = conversation["lines"][i]["text"].strip()  #strip removes the newline character from the sentence
    target = conversation["lines"][i + 1]["text"].strip()
    if question and target:
      qa_pairs.append([question, target])

In [16]:
qa_pairs[0]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you."]

In [17]:
#write these pairs into a new file

datafile = os.path.join("cornell movie-dialogs corpus", "formatted_lines.txt") #newfile
delimiter = "\t"

#use a codec to unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("Writing to the new file")
with open(datafile, "w", encoding = 'utf-8') as outfile:
  writer = csv.writer(outfile, delimiter = delimiter)
  for pair in qa_pairs:
    writer.writerow(pair)
print("Done writing to the new file")    

Writing to the new file
Done writing to the new file


In [18]:
#Visualizing some lines in raw form

datafile = os.path.join("cornell movie-dialogs corpus", "formatted_lines.txt")
with open(datafile, "rb") as file:
  lines = file.readlines()

for line in lines[:8]:
  print(line)  

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\n"
b'Why?\tUnsolved myster

In [19]:
PAD_token = 0 #to pad short sentences
SOS_token = 1
EOS_token = 2

class Vocabulary:
  def __init__(self, name):  #here name is the name of corpus
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token : "PAD", SOS_token : "SOS", EOS_token : "EOS"}
    self.num_words = 3

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1

    else:
      self.word2count[word] += 1

  def trim(self, count_min):
    keepwords = []
    for k, v in self.word2count.items():
      if v >= count_min:
        keepwords.append(k)

    print('Keep words {} / {} = {:.4f}'.format(len(keepwords), len(self.word2index), len(keepwords) / len(self.word2index)))

    #reinitialize dicts to store only the keepwords
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token : "PAD", SOS_token : "SOS", EOS_token : "EOS"}
    self.num_words = 3

    for word in keepwords:
      self.addWord(word)             

In [20]:
#convert unicode to ascii - this removes all the accents etc to make it easier for the nw

def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn') #Mn is a non marking character

In [21]:
def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  #replace any of (.!?) with a space and the special character, \1 refers to the first bracketed group (.!?)
  #r" \1 is so that the \ is not misunderstood as a python keyword
  s = re.sub(r"([.!?])", r" \1", s)

  #+means more than 1
  s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)

  #remove the extra whitespaces
  s = re.sub(r"\s+", r" ", s).strip()
  return s

In [22]:
normalizeString("aha?what1are you")

'aha ?what are you'

In [23]:
datafile = os.path.join("cornell movie-dialogs corpus", "formatted_lines.txt")

print("Opened the file and processing all the text")

lines = open(datafile, encoding = 'utf-8').read().strip().split('\n')  #note that strip is for removing indents spaces and escape seqs and split is used to form a clear split between the lines

#note that \n separates each of the lines and \t separates the pairs of Q and A
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print("Done reading")

voc = Vocabulary(name = "cornell movie-dialogs corpus")

Opened the file and processing all the text
Done reading


In [24]:
pairs

[['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
  'well i thought we d start with pronunciation if that s okay with you .'],
 ['well i thought we d start with pronunciation if that s okay with you .',
  'not the hacking and gagging and spitting part . please .'],
 ['not the hacking and gagging and spitting part . please .',
  'okay . . . then how bout we try out some french cuisine . saturday ? night ?'],
 ['you re asking me out . that s so cute . what s your name again ?',
  'forget it .'],
 ['no no it s my fault we didn t have a proper introduction', 'cameron .'],
 ['cameron .',
  'the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .'],
 ['the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .',
  'seems like she could get a date easy enough . . .'],
 ['why ?

In [25]:
#This is to remove the sentences that have length beyond a certain limit

MAX_LENGTH = 10
def filter_pair(p):
  #p is a pair with Q and A that returns true or false
  return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

def filter_pairs(pairs):
  return [pair for pair in pairs if filter_pair(pair)]


In [26]:
pairs = [pair for pair in pairs if len(pair) > 1]
print("There are {} pairs of Q and A at first".format(len(pairs)))
pairs = filter_pairs(pairs)
print("After filtering there are {} pairs of Q&A".format(len(pairs)))


There are 221282 pairs of Q and A at first
After filtering there are 64271 pairs of Q&A


In [27]:
#add the Q and the A to the vocab object
for pair in pairs:
  voc.addSentence(pair[0])
  voc.addSentence(pair[1])

print("Counted words: ", voc.num_words)
for pair in pairs[:10]:
  print(pair)  

Counted words:  18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [28]:
MinCount = 3

def trimRareWords(voc, pairs, MinCount):

  voc.trim(MinCount)
  #reinitialize

  keep_pairs = []
  for pair in pairs:
    input_sentence = pair[0]
    output_sentence = pair[1]
    keep_input = True
    keep_output = True

    for word in input_sentence.split(' '):
      if word not in voc.word2index:
        keep_input = False   
        break

    for word in output_sentence.split(' '):
      if word not in voc.word2index:
        keep_output = False
        break

    if keep_input and keep_output:
      keep_pairs.append(pair)

  print("trimmed from {} pairs to {} pairs".format(len(pairs), len(keep_pairs)))
  print("Percentage of total retained: {:.4f}".format(len(keep_pairs) / len(pairs)))
  return keep_pairs

pairs = trimRareWords(voc, pairs, MinCount)      
              


Keep words 7823 / 18005 = 0.4345
trimmed from 64271 pairs to 53165 pairs
Percentage of total retained: 0.8272


In [29]:
def indexes_from_sentences(voc, sentence):
  return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token] 

In [30]:
pairs[1][0]

'you have my word . as a gentleman'

In [31]:
indexes_from_sentences(voc, pairs[1][0])

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [32]:
 #test and see the changes
inp = []
output = []
for pair in pairs[:10]:
  inp.append(pair[0])
  output.append(pair[1])
print(inp)  
print(len(inp))
indexes = [indexes_from_sentences(voc, sentence) for sentence in inp]

indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow']
10


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [58, 2]]

In [33]:
def zeroPadding(l, fillvalue = 0):
  return list(itertools.zip_longest(*l, fillvalue = fillvalue))

In [34]:
a = [[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [58, 2]]

In [35]:
list(itertools.zip_longest(*a, fillvalue = 0))  #here the cols represent one sentence of dim maxlen and one row is a batch which is fed into each timestep

[(3, 7, 16, 8, 33, 35, 42, 47, 50, 58),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [36]:
single_length = [len(s) for s in indexes]
max(single_length)

10

In [37]:
single_length

[3, 9, 3, 5, 6, 10, 2, 8, 5, 2]

In [38]:
#test the funcs
test_result = zeroPadding(indexes)
print(len(test_result))
test_result

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 58),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [39]:
def binary_matrix(l, value = 0): #the pad tokens are pushed to 0 and the other values are pushed to 1
  m = []
  for i, seq in enumerate(l):
    m.append([])
    for token in seq:
      if token == PAD_token:
        m[i].append(0)

      else:
        m[i].append(1)
  return m        

In [40]:
binary_result = binary_matrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [41]:
def inputVar(l, voc):
  indexes_batch = [indexes_from_sentences(voc, sentence) for sentence in l]
  lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  padVar = torch.FloatTensor(padList)
  return padVar, lengths

In [42]:
def outputVar(l, voc):
  indexes_batch = [indexes_from_sentences(voc, sentence) for sentence in l]
  max_target_len = max([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  mask = binary_matrix(padList)
  mask = torch.ByteTensor(mask)
  padVar = torch.FloatTensor(padList)
  return padVar, mask, max_target_len

In [43]:
#return a batch of items for training

def batch2TrainData(voc, pair_batch):
  #sort the questions in desc order of lengths
  pair_batch.sort(key = lambda x: len(x[0].split(" ")), reverse = True)
  input_batch, output_batch = [], []
  for pair in pair_batch:
    input_batch.append(pair[0])
    output_batch.append(pair[1])
  inp, lengths = inputVar(input_batch, voc)
  output, mask, max_target_len = outputVar(output_batch, voc)
  return inp, lengths, output, mask, max_target_len

In [73]:
small_batch_size = 5 
batch = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_var, length_input, target_var, mask, max_target_len = batch

print("Input Variable: ")
print(input_var)
print(input_var.type)

print("Lengths of inputs ")
print(length_input)

print("Target variable: ")
print(target_var)

print("Mask: ")
print(mask)

print("max_target_len: ")
print(max_target_len)

Input Variable: 
tensor([[2.7300e+02, 1.9700e+02, 2.5000e+01, 2.0000e+02, 1.4720e+03],
        [1.8000e+02, 1.1700e+02, 8.6000e+01, 3.0370e+03, 4.0000e+00],
        [9.6000e+01, 4.7000e+01, 3.0200e+02, 4.0000e+00, 2.0000e+00],
        [2.0470e+03, 7.6000e+01, 1.4040e+03, 2.0000e+00, 0.0000e+00],
        [4.0000e+00, 3.1610e+03, 7.6000e+01, 0.0000e+00, 0.0000e+00],
        [1.5800e+02, 6.6000e+01, 4.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.6000e+01, 2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0820e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])
<built-in method type of Tensor object at 0x7fbefd3408b8>
Lengths of inputs 
tensor([10,  7,  7,  4,  3])
Target variable: 
tensor([[2.5270e+03, 1.0100e+02, 2.5000e+01, 1.2200e+02, 5.0000e+01],
        [2.1900e+02, 3.7000e+01, 2.4000e+01, 3.6200e+03, 7.5000e+01],
        [3.7000e+0

In [65]:
class EncoderRNN(nn.Module):
  def __init__(self, hidden_size, embedding, n_layers = 1, dropout = 0):
    super(EncoderRNN, self).__init__()
    self.n_layers = n_layers
    self.hidden_size = hidden_size
    self.embedding = embedding
    #GRU (with a packed padded seq is used - BIDIRECTIONAL) - ip size is same as hidden state (embed dims = hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout = (0 if n_layers == 1 else dropout), bidirectional = True)

  def forward(self, input_seq, input_lengths, hidden = None):
    embedded = self.embedding(input_seq)
    #pack-padded batch 
    packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
    outputs, hidden = self.gru(packed, hidden)
    #unpack the padding
    outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
    #sum up the bidirectional ops forward and reverse
    outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]

    return outputs, hidden   
    #shapes - 
    # input sequence = max_length, batch_size
    # hidden state = n_layers x num_directions(2) , batch_size, hidden_size
    # outputs = max_length, batch_size, hidden_size  

    # packed padded sequence will remove all the zero padding and feed it into the nw with faster computation

In [66]:
#Luong attention layer
class Attn(nn.Module):
  def __init__(self, method, hidden_size):
    super(Attn, self).__init__()
    self.method = method
    self.hidden_size = hidden_size

  def dot_score(self, hidden, encoder_output):
    return torch.sum(hidden * encoder_output, dim = 2)  #hidden = (1, batchdims, hiddensize) (as only 1 GRU)  , encoder op = (max_len, bdims, hsize) - element wise * will broadcast 1 into seqlength, and sum across batch_dims

  def forward(self, hidden, encoder_outputs):  
    attn_energies = self.dot_score(hidden, encoder_outputs)  #after summing across dim2 - dim = (max_len, batch_size)
    attn_energies = attn_energies.t()  #to find softmax for every sample (every row - every batch)

    return F.softmax(attn_energies, dim = 1).unsqueeze(1)  #batchsize, 1, max_length 

In [67]:
#Luong attention decoder layer

class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers = 1, dropout = 0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.attn_model = attn_model
    self.embedding = embedding
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    self.dropout = dropout

    #Other layers
    self.embedding_dropout = nn.Dropout(dropout)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout = (0 if n_layers == 1 else dropout)) #note here that ip and hidden layer size is same
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)

    self.attn = Attn(method = attn_model, hidden_size = hidden_size)


  def forward(self, input_step, last_hidden, encoder_outputs): #seq_len == max_length
    #input step is one sample from the input sequence of size seq_len, batch_size -> (1, batch_size)
    #last hidden is the prev hidden state of the GRU (dims = (n_layers * n_directions, batch_size, hidden_size))
    #encoder outputs have a dims of (seq_len, batch_size, hidden_size)  

    #running this step one word at a time
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    #through unidirectional GRU (decoder)
    rnn_output, hidden = self.gru(embedded, last_hidden)
    #attention weights can be calculated from the gru op
    attn_weights = self.attn(rnn_output, encoder_outputs)
    #we need to * the encoder ops and the attn weights to get the context vector
    # we use bmm - batch Multiplication - (batch_size , 1, max_length) * (batch_size * max_length, hidden_size) = keep every batch as it is = (batch_size, 1, hidden_size)
    context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
    #concat this with gru op
    #preprocess it first
    #remove the 1 dims
    rnn_output = rnn_output.squeeze(0)
    context = context.squeeze(1)
    #concat along columns - (batch_size, 2*hidden_size)
    concat_input = torch.cat((rnn_output, context), 1)
    concat_output = torch.tanh(self.concat(concat_input))
    #luang eqn - pass through one more dense layer with op = vocab size and take softmax for preds of what the next word is
    output = self.out(concat_output)
    output = F.softmax(output, dim = 1)
    #return the op and GRU hidden state for next step
    return output, hidden
    #output = (batch_size, vocab_size) - prediction for what the next word is
    #hidden - (n_layers * n_directions, batch_size, hidden_layers)

In [68]:
#Finally, TRAINING
#Loss function - Negative log likelihood loss is used
def maskNLLLoss(decoder_out, target, mask):
  num_total = mask.sum #how many elements we need to consider ( we take only non zero elements and here the mask shows if the elem is 0 or not)
  target = target.view(-1, 1)
  #decoder out shape is (batch_size, vocab_size) - target is (batch_size, 1)
  gathered_tensor = torch.gather(decoder_out, 1, target)
  #Calculate the neg log likelihood loss
  crossEntropy = -torch.log(gathered_tensor)
  #Select the non-zero elements
  loss = crossEntropy.masked_select(mask)
  #Calculate the mean of the loss
  loss = loss.mean()
  loss = loss.to(device) #pass to gpu
  return loss, num_total.item()

In [69]:
#Visual of what is happening in the loss function
#decoder out shape is (batch_size, vocab_size) - target is (batch_size, 1)
dec_o = torch.rand(5, 7)
dec_o = F.softmax(dec_o, dim = 1)
targ = torch.tensor([2, 1, 5, 4, 0], dtype = torch.long)
targ = targ.view(-1, 1)
mask = torch.tensor([1, 0, 1, 1, 0], dtype = torch.uint8)
print(dec_o)
print(targ)
gath_ten = torch.gather(dec_o, 1, targ)
print(gath_ten)
print(gath_ten.shape)
crossEntropy = -torch.log(gath_ten)
print("Cross Entropy: ", crossEntropy)
mask = mask.unsqueeze(1)
loss = crossEntropy.masked_select(mask)
print("Loss: ")
print(loss)
print(loss.shape)
print("Sum of mask elements (ones we are considering to calc loss", mask.sum())
print("Mean of loss: ", loss.mean())
print("Mean Loss of the CE loss before masking: ", crossEntropy.mean())


tensor([[0.2330, 0.1265, 0.0924, 0.1339, 0.1296, 0.1619, 0.1226],
        [0.1090, 0.0973, 0.1550, 0.1861, 0.1125, 0.2308, 0.1093],
        [0.1047, 0.0853, 0.1259, 0.1771, 0.0914, 0.1915, 0.2240],
        [0.1505, 0.1329, 0.1691, 0.1853, 0.1727, 0.0757, 0.1138],
        [0.1652, 0.0834, 0.1726, 0.1786, 0.0842, 0.1515, 0.1645]])
tensor([[2],
        [1],
        [5],
        [4],
        [0]])
tensor([[0.0924],
        [0.0973],
        [0.1915],
        [0.1727],
        [0.1652]])
torch.Size([5, 1])
Cross Entropy:  tensor([[2.3819],
        [2.3302],
        [1.6529],
        [1.7562],
        [1.8005]])
Loss: 
tensor([2.3819, 1.6529, 1.7562])
torch.Size([3])
Sum of mask elements (ones we are considering to calc loss tensor(3)
Mean of loss:  tensor(1.9303)
Mean Loss of the CE loss before masking:  tensor(1.9843)


/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:29: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [75]:
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable.item, lengths, target_variable, mask, max_target_len = batches

print("Input variable shape: ")
print(input_variable.shape)
print("Lengths shape ")
print(lengths.shape)
print("Target variable shape: ")
print(target_variable.shape)
print("Mask shape : ")
print(mask.shape)
print("max_target_len: ")
print(max_target_len)

#params
hidden_size = 500 #each gru has 500 nodes
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.2
attn_model = 'dot'  #dot product variant - easy
embedding = nn.Embedding(voc.num_words, hidden_size)

#Encoder and decoder
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
encoder = encoder.to(device)
decoder = decoder.to(device)
#activ dropout by indicating train phase

encoder.train()
decoder.train()

#optims initialization
encoder_optimizer = optim.Adam(encoder.parameters(), lr = 0.0001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = 0.0001)
encoder_optimizer.zero_grad()
decoder_optimizer.zero_grad()

input_variable = input_variable.to(device)
lengths = lengths.to(device)
target_variable = target_variable.to(device)
mask = mask.to(device)

loss = 0
print_losses = []
n_totals = 0 #number of included terms

encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
print("Shape of the encoder outputs: ", encoder_outputs.shape)
print("Last encoder hidden state dims ", encoder_hidden.shape)

decoder_input = torch.LongTensor([[SOS_token for _ in range(small_batch_size)]])
decoder_input = decoder_input.to(device)
print("Initial Decoder input shape", decoder_input.shape)
print(decoder_input)

#final enc state is the dec hidden state
decoder_hidden = encoder_hidden[:decoder.n_layers]
print("Initial decoder hidden state shape: ", decoder_hidden.shape)
print(decoder_hidden.shape)
print("\n")
print("-----------------------------------------")
print("Actions in one time-step of the GRU")
print("-----------------------------------------")
print("\n")


#assume teacher forcing principle
for t in range(max_target_len):  #max timesteps is the max seq length
  decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
  print("Decoder output shape: ", decoder_output.shape)
  print("Decoder hidden shape: ", decoder_hidden.shape)

  #by forcing, the next input is the current target
  print("target variable at current timestep before reshaping: ", target_variable[t])
  print("target variable shape at current timestep before reshaping: ", target_variable[t].shape)
  print("target variable at current timestep on reshaping (decoder ip): ", decoder_input.shape)

  #Calc the loss using the mask and accumulate it
  print("mask at the current timestep: ", mask[t])
  print("mask shape at the current timestep: ", mask[t].shape)
  mask_loss, n_total = maskNLLLoss(decoder_output, target_variable[t], mask[t])
  print("Mask Loss: ", mask_loss)
  print("Total: ", n_total)
  loss += mask_loss
  print_losses.append(mask_loss.item() * n_total)
  print(print_losses)
  n_totals += n_total
  print(n_totals)
  encoder_optimizer.step()
  decoder_optimizer.step()
  returned_loss = sum(print_losses) / n_totals
  print("Returned Loss: ", returned_loss)
  print("\n")
  print("------------ONE time step OVER-------------")
  print("\n")


Input variable shape: 
torch.Size([10, 5])
Lengths shape 
torch.Size([5])
Target variable shape: 
torch.Size([10, 5])
Mask shape : 
torch.Size([10, 5])
max_target_len: 
10


RuntimeError: ignored

In [76]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer,
          decoder_optimizer, batch_size, clip, max_length = MAX_LENGTH):
  
  #zero grads
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  #Set device options
  input_variable = input_variable.to(device)
  lengths = lengths.to(device)
  target_variable = target_variable.to(device)
  mask = mask.to(device)  

  loss = 0
  print_losses = []
  n_totals = 0

  encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
  decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
  decoder_input = decoder_input.to(device)

  decoder_hidden = encoder_hidden[:decoder.n_layers]

      # Determine if we are using teacher forcing this iteration
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  # Forward batch of sequences through decoder one time step at a time
  if use_teacher_forcing:
      for t in range(max_target_len):
          decoder_output, decoder_hidden = decoder(
              decoder_input, decoder_hidden, encoder_outputs
          )
          # Teacher forcing: next input is current target
          decoder_input = target_variable[t].view(1, -1)
          # Calculate and accumulate loss
          mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
          loss += mask_loss
          print_losses.append(mask_loss.item() * nTotal)
          n_totals += nTotal
  else:
      for t in range(max_target_len):
          decoder_output, decoder_hidden = decoder(
              decoder_input, decoder_hidden, encoder_outputs
          )
          # No teacher forcing: next input is decoder's own current output
          _, topi = decoder_output.topk(1)
          decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
          decoder_input = decoder_input.to(device)
          # Calculate and accumulate loss
          mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
          loss += mask_loss
          print_losses.append(mask_loss.item() * nTotal)
          n_totals += nTotal

  # Perform backpropatation
  loss.backward()

  # Clip gradients: gradients are modified in place
  _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
  _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

  # Adjust model weights
  encoder_optimizer.step()
  decoder_optimizer.step()

  return sum(print_losses) / n_totals









In [88]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, n_iteration, batch_size, print_every, save_every, clip, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # # Save checkpoint
        # if (iteration % save_every == 0):
        #     directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
        #     if not os.path.exists(directory):
        #         os.makedirs(directory)
        #     torch.save({
        #         'iteration': iteration,
        #         'en': encoder.state_dict(),
        #         'de': decoder.state_dict(),
        #         'en_opt': encoder_optimizer.state_dict(),
        #         'de_opt': decoder_optimizer.state_dict(),
        #         'loss': loss,
        #         'voc_dict': voc.__dict__,
        #         'embedding': embedding.state_dict()
        #     }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [89]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [90]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [91]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [92]:
#TRAIN - 
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, n_iteration, batch_size,
           print_every, save_every, clip, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


RuntimeError: ignored

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
# evaluateInput(encoder, decoder, searcher, voc)